#### Considerando la escasez de recursos disponibles para el deployment gratuito de nuestra API en Render, hemos decidido crear este Jupyter Notebook para preparar los datasets específicos necesarios para cada endpoint. En este Notebook, se realizarán cálculos y operaciones pesadas que podrían ralentizar nuestras consultas en la API. Además, esta herramienta nos permitirá ahorrar la carga de datasets completos que no son necesarios para el MVP. De esta manera, podremos optimizar el rendimiento de nuestra API y ofrecer un servicio más eficiente a nuestros usuarios.

In [1]:
import pandas as pd

### Importamos los datasets generados ad hoc en nuestras anteriores ETL

In [2]:
df_games_complete=pd.read_parquet('processed_data/games.parquet')

In [3]:
df_items_complete=pd.read_parquet('processed_data/items.parquet')

In [4]:
df_reviews_complete=pd.read_parquet('processed_data/reviews.parquet')

### Preparacion de dataset para la funcion PlayTimeGenre

#### def PlayTimeGenre( genero : str ): 
Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: 
{"Año de lanzamiento con más horas jugadas para Género X" : 2013}


#### #Se crean dataframes solo con las columnas que vamos a utilizar

In [5]:
df_games=df_games_complete[['genres','release_year','id']]

In [6]:
df_items=df_items_complete[['item_id','playtime_forever']]

### #Se realiza un explode de los generos

In [7]:
df_games=df_games.explode('genres',ignore_index=False,)

### #Se hace un join entre df_games y df_items usando 'id' y 'item_id'

In [ ]:
df_games['id'] = df_games['id'].astype(int)
df_items['item_id'] = df_items['item_id'].astype(int)
df_joined = pd.merge(df_games, df_items, left_on='id', right_on='item_id', how='inner')

In [9]:

df_joined 

,genres,release_year,id,item_id,playtime_forever
0,Action,1997.0,282010,282010,5.0
1,Action,1997.0,282010,282010,0.0
2,Action,1997.0,282010,282010,0.0
3,Action,1997.0,282010,282010,0.0
4,Action,1997.0,282010,282010,13.0
...,...,...,...,...,...
10222881,Action,2004.0,80,80,0.0
10222882,Action,2004.0,80,80,0.0
10222883,Action,2004.0,80,80,0.0
10222884,Action,2004.0,80,80,9.0


### #Se agrupa por género y año, sumando las horas jugadas

In [10]:
df_grouped = df_joined.groupby(['genres', 'release_year'])['playtime_forever'].sum().reset_index()
df_grouped

,genres,release_year,playtime_forever
0,Action,1983.0,3473.0
1,Action,1984.0,384.0
2,Action,1988.0,16001.0
3,Action,1989.0,607.0
4,Action,1990.0,18335.0
...,...,...,...
338,Web Publishing,2013.0,333678.0
339,Web Publishing,2014.0,33641.0
340,Web Publishing,2015.0,348673.0
341,Web Publishing,2016.0,136.0


### #Se encuentra el índice del máximo de horas jugadas para cada género

In [11]:
idx = df_grouped.groupby('genres')['playtime_forever'].idxmax()
idx

genres
Action                    26
Adventure                 59
Animation & Modeling      71
Audio Production          75
Casual                   104
Design & Illustration    109
Early Access             116
Education                124
Free to Play             141
Indie                    158
Massively Multiplayer    178
Photo Editing            186
RPG                      208
Racing                   235
Simulation               257
Software Training        273
Sports                   288
Strategy                 314
Utilities                324
Video Production         333
Web Publishing           337
Name: playtime_forever, dtype: int64

### #Se filtra df_grouped usando los índices encontrados

In [12]:
df_max_year_per_genre = df_grouped.loc[idx]
df_max_year_per_genre

,genres,release_year,playtime_forever
26,Action,2012.0,1.085803e+09
59,Adventure,2011.0,2.218807e+08
71,Animation & Modeling,2015.0,1.345913e+06
75,Audio Production,2014.0,4.554630e+05
104,Casual,2015.0,8.171851e+07
109,Design & Illustration,2012.0,1.930339e+06
116,Early Access,2013.0,1.189029e+08
124,Education,2013.0,3.405670e+05
141,Free to Play,2013.0,1.462138e+08
158,Indie,2006.0,4.461153e+08


### #Se crea un dataframe solo con las columnas necesarias para nuestra consulta y se guarda para su posterior uso

In [13]:
df_function_PlayTimeGenre=df_max_year_per_genre[['genres','release_year']]

In [15]:
df_function_PlayTimeGenre.to_parquet('API/play_time_genre.parquet')

### Preparacion de dataset para la funcion UserForGenre

#### def UserForGenre( genero : str ): 
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: 
{"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}


### #Se realiza un explode de los generos

In [16]:
df_games=df_games.explode('genres',ignore_index=False,)

### #Se hace un join entre df_games y df_items usando 'id' y 'item_id'

In [18]:
df_games['id'] = df_games['id'].astype(int)
df_items['item_id'] = df_items['item_id'].astype(int)
df_joined = pd.merge(df_games, df_items, left_on='id', right_on='item_id', how='inner')
df_joined

C:\Users\Usr\AppData\Local\Temp\ipykernel_15544\1284996760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items['item_id'] = df_items['item_id'].astype(int)


,genres,release_year,id,item_id,playtime_forever
0,Action,1997.0,282010,282010,5.0
1,Action,1997.0,282010,282010,0.0
2,Action,1997.0,282010,282010,0.0
3,Action,1997.0,282010,282010,0.0
4,Action,1997.0,282010,282010,13.0
...,...,...,...,...,...
10222881,Action,2004.0,80,80,0.0
10222882,Action,2004.0,80,80,0.0
10222883,Action,2004.0,80,80,0.0
10222884,Action,2004.0,80,80,9.0


### #Se agrupa por género, usuario y año, sumando las horas jugadas

In [20]:
df_joined

,genres,release_year,id,item_id,playtime_forever
0,Action,1997.0,282010,282010,5.0
1,Action,1997.0,282010,282010,0.0
2,Action,1997.0,282010,282010,0.0
3,Action,1997.0,282010,282010,0.0
4,Action,1997.0,282010,282010,13.0
...,...,...,...,...,...
10222881,Action,2004.0,80,80,0.0
10222882,Action,2004.0,80,80,0.0
10222883,Action,2004.0,80,80,0.0
10222884,Action,2004.0,80,80,9.0


In [21]:
df_grouped = df_joined.groupby(['genres','id', 'release_year'])['playtime_forever'].sum().reset_index()
df_grouped

,genres,id,release_year,playtime_forever
0,Action,10,2000.0,17107858.0
1,Action,20,1999.0,960524.0
2,Action,30,2003.0,756375.0
3,Action,40,2001.0,154424.0
4,Action,50,1999.0,726545.0
...,...,...,...,...
21623,Web Publishing,400040,2007.0,11058.0
21624,Web Publishing,407460,2015.0,0.0
21625,Web Publishing,478960,2016.0,135.0
21626,Web Publishing,483950,2016.0,1.0


### #Se calcula el total de horas jugadas por usuario y genero

In [23]:
df_total_hours = df_grouped.groupby(['genres', 'id'])['playtime_forever'].sum().reset_index()
df_total_hours

,genres,id,playtime_forever
0,Action,10,17107858.0
1,Action,20,960524.0
2,Action,30,756375.0
3,Action,40,154424.0
4,Action,50,726545.0
...,...,...,...
21623,Web Publishing,400040,11058.0
21624,Web Publishing,407460,0.0
21625,Web Publishing,478960,135.0
21626,Web Publishing,483950,1.0


### #Se busca al usuario con mas horas jugadas por genero

In [24]:
idx = df_total_hours.groupby('genres')['playtime_forever'].idxmax()

idx

genres
Action                      22
Adventure                 4101
Animation & Modeling      6911
Audio Production          6934
Casual                    7681
Design & Illustration     9267
Early Access              9343
Education                 9833
Free to Play              9924
Indie                    10358
Massively Multiplayer    15749
Photo Editing            15956
RPG                      16111
Racing                   17634
Simulation               17874
Software Training        19204
Sports                   19290
Strategy                 19615
Utilities                21531
Video Production         21601
Web Publishing           21610
Name: playtime_forever, dtype: int64

In [25]:
df_user_max_hours = df_total_hours.loc[idx]
df_user_max_hours

,genres,id,playtime_forever
22,Action,730,775918724.0
4101,Adventure,105600,152997644.0
6911,Animation & Modeling,365670,1148407.0
6934,Audio Production,228180,390165.0
7681,Casual,304930,50333977.0
9267,Design & Illustration,220700,1930339.0
9343,Early Access,252490,80110135.0
9833,Education,227240,175684.0
9924,Free to Play,230410,122726853.0
10358,Indie,4000,441871026.0


### #Se filtra el dataframe completo para obtener los registros del usuario con mas horas jugadas para cada genero

In [27]:
df_max_user_per_genre = pd.merge(df_joined, df_user_max_hours[['genres', 'id']], on=['genres', 'id'], how='inner')
df_max_user_per_genre

,genres,release_year,id,item_id,playtime_forever
0,Indie,2006.0,4000,4000,412.0
1,Indie,2006.0,4000,4000,68.0
2,Indie,2006.0,4000,4000,2527.0
3,Indie,2006.0,4000,4000,358.0
4,Indie,2006.0,4000,4000,4654.0
...,...,...,...,...,...
329898,RPG,2011.0,105600,105600,378.0
329899,RPG,2011.0,105600,105600,8688.0
329900,RPG,2011.0,105600,105600,128.0
329901,RPG,2011.0,105600,105600,4367.0


### #Se agrupa por género, usuario y año el dataframe filtrado , sumando las horas jugadas

In [28]:
df_max_user_per_genre=df_max_user_per_genre.groupby(['genres','id','release_year'])['playtime_forever'].sum().reset_index()
df_max_user_per_genre

,genres,id,release_year,playtime_forever
0,Action,730,2012.0,775918724.0
1,Adventure,105600,2011.0,152997644.0
2,Animation & Modeling,365670,2015.0,1148407.0
3,Audio Production,228180,2014.0,390165.0
4,Casual,304930,2017.0,50333977.0
5,Design & Illustration,220700,2012.0,1930339.0
6,Early Access,252490,2013.0,80110135.0
7,Education,227240,2013.0,175684.0
8,Free to Play,230410,2013.0,122726853.0
9,Indie,4000,2006.0,441871026.0


### #Se crea un dataframe solo con las columnas necesarias para nuestra consulta y se guarda para su posterior uso

In [29]:
df_function_UserForGenre=df_max_user_per_genre[['genres','release_year','id','playtime_forever']]
df_function_UserForGenre

,genres,release_year,id,playtime_forever
0,Action,2012.0,730,775918724.0
1,Adventure,2011.0,105600,152997644.0
2,Animation & Modeling,2015.0,365670,1148407.0
3,Audio Production,2014.0,228180,390165.0
4,Casual,2017.0,304930,50333977.0
5,Design & Illustration,2012.0,220700,1930339.0
6,Early Access,2013.0,252490,80110135.0
7,Education,2013.0,227240,175684.0
8,Free to Play,2013.0,230410,122726853.0
9,Indie,2006.0,4000,441871026.0


### Se guarda user_for_genre.parquet

In [30]:
df_function_UserForGenre.to_parquet('API/user_for_genre.parquet')

### Generamos el dataframe que se van a utilizar en los endpoints UsersWorstDeveloper, sentiment_analysis y recomendacion_juego

def UsersWorstDeveloper( año : int ): 
Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados 
por usuarios para el año dado. 
(reviews.recommend = False y comentarios negativos)
Ejemplo de retorno:
 [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



def sentiment_analysis( empresa desarrolladora : str ): 
Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave 
y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
Ejemplo de retorno:
 {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}


def UsersRecommend( año : int ): 
Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: 
[{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]


### #Se crea el dataframe solo con las columnas que se van a utilizar y se guarda en formato parquet para su posterior uso.

In [32]:
df_games=df_games_complete[['id','app_name','developer']]
df_games

,id,app_name,developer
0,761140.0,Lost Summoner Kitty,Kotoshiro
1,643980.0,Ironbound,Secret Level SRL
2,670290.0,Real Pool 3D - Poolians,Poolians.com
3,767400.0,弹炸人2222,彼岸领域
4,773570.0,Log Challenge,None
...,...,...,...
32127,773640.0,Colony On Mars,"Nikita ""Ghost_RUS"""
32128,733530.0,LOGistICAL: South Africa,Sacada
32129,610660.0,Russian Roads,Laush Dmitriy Sergeevich
32130,658870.0,EXIT 2 - Directions,"xropi,stev3ns"


### Guardamos en parquet

In [33]:
df_games.to_parquet('API/games_endpoints.parquet')

In [34]:
df_reviews_complete

,user_id,item_id,recommend,posted_year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,1
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,1
...,...,...,...,...,...
58426,76561198312638244,70,True,2016,2
58427,76561198312638244,362890,True,2016,2
58428,LydiaMorley,273110,True,2016,1
58429,LydiaMorley,730,True,2016,2


In [35]:
df_reviews_complete.to_parquet('API/reviews.parquet')